# 1. Memória: referências, mutabilidade, e coleta de lixo

Este notebook contém exemplos do livro [_Fluent Python, Second Edition_](https://www.fluentpython.com/) .

## Referências e coleções

É comum falarmos de coleções *mutáveis* ou *imutáveis* em Python. Coleções mutáveis permitem retirar, acrescentar, ou substituir itens, mas as imutáveis não permitem.

Sequências são uma categoria específica de coleção na qual os itens se organizam um atrás do outro e podem ser acessados por índices a partir de zero.

Os tipos `tuple` e `array` representam duas categorias de sequências:

* *Sequência container*: pode conter itens de diferentes tipos ao mesmo tempo, inclusive outros containers. Ex: `tuple`, `list`, `collections.deque`etc.

* *Sequência simples*: pode conter itens de um tipo simples pré-determinado. Ex: `str`, `bytes`, `array.array`.

Veja essas duas sequências, uma tupla com 3 elementos e um array com 3 elementos:

In [1]:
t = 9.46, 'cat', [2.08, 4.29]
t

(9.46, 'cat', [2.08, 4.29])

In [2]:
from array import array
a = array('d', [9.46, 2.08, 4.29])
a

array('d', [9.46, 2.08, 4.29])

![tuple x array](fig2-1.png)


A figura acima mostra de forma simplificada os valores da tupla `t` e do array `a`.

Uma *sequência container* não armazena os itens diretamente em seu espaço de memória, mas armazena referências aos itens, que estão em outras partes da memória. Em contraste, uma *sequência simples* armazena os valores dos itens diretamente em uma área contígua de posições de memória, e não como objetos separados.



## Variáveis não são caixas

Alguns professores sugerem que "variáveis são como caixas".
Essa analogia faz sentido em linguagens como C ou Pascal,
mas não funciona em Python, Java, JS, Ruby,
ou qualquer outra linguagem onde variáveis são referências a objetos.


Veja um código que quebra a ideia de que variáveis são caixas:

In [3]:
a = [1, 2, 3]
b = a
a.append(4)
a, b

([1, 2, 3, 4], [1, 2, 3, 4])

Se `a` e `b` são como "caixas", como explicar o que aconteceu? Mas se você imaginar que `a` e `b` são duas etiquetas coladas no mesmo objeto, faz sentido.

![Fig. 6.1](https://raw.githubusercontent.com/fluentpython/images/master/object-refs/var-boxes-x-labels.png)



## Objetos são criados antes de variáveis

O código abaixo prova que o lado direito da instrução `x = Trem()` é executado primeiro,
e só depois acontece a atribuição.

Primeiro a classe `Trem`, que apenas exibe uma mensagem quando uma instância é criada.

In [4]:
class Trem:
    def __init__(self):
        print(f'Trem id: {id(self):#x}')
        
x = Trem()

Trem id: 0x103911cd0


In [5]:
x

In [6]:
'x' in globals()

True

In [7]:
#y = Trem() * Trem()  # isso vai gerar um erro

In [8]:
'y' in globals()

False

In [9]:
#y  # isso vai gerar outro erro

## Identidade, igualdade, e aliases

> **NOTA:** A palavra inglesa *alias* pode ser traduzida como *apelido* ou *alcunha*.

Pelé era o apelido do jogador Edson Arantes do Nascimento.
Pelé não é apenas igual a Edson, os dois nomes se referem à mesma pessoa.

Veja esta ideia em Python:

In [10]:
edson = dict(nome='Edson Arantes do Nascimento', ano=1940)
edson

{'nome': 'Edson Arantes do Nascimento', 'ano': 1940}

In [11]:
pelé = edson
pelé is edson

True

In [12]:
id(pelé), id(edson)

(4360590400, 4360590400)

In [13]:
pelé['gols'] = 1283
edson

{'nome': 'Edson Arantes do Nascimento', 'ano': 1940, 'gols': 1283}

Agora imagine um impostor, tentando se passar por Pelé.
O impostor alega que tem os mesmos dados:

In [14]:
impostor = {'nome': 'Edson Arantes do Nascimento', 'ano': 1940, 'gols': 1283}

In [15]:
impostor is pelé

False

In [16]:
impostor == pelé

True

In [17]:
impostor is not pelé

True

## Como escolher entre `==` ou `is`

É muito mais comum a gente comparar o *valor* de dois objetos do que a *identidade* deles.

Para comparar valores, sempre use `==`. Quase sempre é o que você quer.

O caso mais comum de uso de `is` é comparar com objetos únicos (singletons),
geralmente usados como sinalizadores ou sentinelas.

Quando um objeto é único é mais eficiente verificar a identidade do que a igualdade,
porque o operador `==` pode ser sobrecarregado implementando o método `__eq__`,
portanto toda vez que aparece `==`, o interpretador precisa verificar a presença do método `__eq__`.

Mas o operador `is` não pode ser sobrecarregado.
Ele é implementado em C simplesmente comparando o *id* dos objetos.

Exemplo:

In [18]:
from unicodedata import name

for código in range(0x30):
    car = chr(código)
    nome = name(car, None)
    if nome is None:
        continue
    print(f'U+{código:04x}\t{car}\t{nome}')

U+0020	 	SPACE
U+0021	!	EXCLAMATION MARK
U+0022	"	QUOTATION MARK
U+0023	#	NUMBER SIGN
U+0024	$	DOLLAR SIGN
U+0025	%	PERCENT SIGN
U+0026	&	AMPERSAND
U+0027	'	APOSTROPHE
U+0028	(	LEFT PARENTHESIS
U+0029	)	RIGHT PARENTHESIS
U+002a	*	ASTERISK
U+002b	+	PLUS SIGN
U+002c	,	COMMA
U+002d	-	HYPHEN-MINUS
U+002e	.	FULL STOP
U+002f	/	SOLIDUS


Sentinelas são valores especiais usados em filas ou sockets
para sinalizar o fim de uma sequência de valores.

Exemplo:

In [19]:
from queue import SimpleQueue

FIM_DA_SÉRIE = object()
BICICLETA = object()

FIM_DA_SÉRIE == BICICLETA

False

In [20]:
FIM_DA_SÉRIE is not BICICLETA

True

In [21]:
def preparar():
    fila = SimpleQueue()
    for letra in 'ABC':
        fila.put(letra)
    fila.put(FIM_DA_SÉRIE)
    return fila

fila = preparar()
    
while (item := fila.get()) is not FIM_DA_SÉRIE:
    print(item)  

A
B
C


> **NOTA:** Porque não usar um laço `for` para percorrer uma fila?<br>Porque a operação `fila.get()` bloqueia quando não há itens na fila. Filas são muito usadas para sincronizar e trocar dados entre threads.

----

### 🤔 Operador Morsa `:=`

O laço **`while`** acima usa o "operador morsa" `:=` introduzido no Python 3.8.

O exemplo acima antes do Python 3.8 seria escrito assim:

In [22]:
fila = preparar()

while True:
    item = fila.get()
    if item is FIM_DA_SÉRIE:
        break
    print(item)

A
B
C


Chama-se "operador morsa" por causa do emoticon `:=` que representa uma morsa:

<img src="https://upload.wikimedia.org/wikipedia/commons/c/ce/Noaa-walrus22.jpg" width="300">

----

## Afinal, tuplas são mesmo imutáveis?

A melhor resposta é: **depende**.

Uma tupla é um _container_: uma estrutura de dados que contém referências a outros objetos.
Outros containers são listas, dicts, sets, e filas.

> Em contraste, `str`, `bytes` e `array` são _flat coleções planas_: armazenam diretamente seus dados, e não referências a outros objetos.

Em uma tupla, as referências são imutáveis.
Ou seja, uma vez criada a tupla, ela sempre apontará para a mesma sequência de objetos.

Porém, se qualquer objeto contido na tupla for mutável,
e seu valor for alterado, isso mudará também o valor da tupla!

![tupla-com-lista](fig2-4.png)

In [23]:
a = (10, 'alpha', [1, 2])
b = (10, 'alpha', [1, 2])
a == b  # valores iguais

True

In [24]:
a is b  # identidades diferentes

False

In [25]:
id(b[-1])  # id da lista que é o último item da tupla b

4360618816

In [26]:
b[-1].append(99)  # alterando a lista t1[-1]
a == b  # agora as tuplas tem valores diferentes

False

In [27]:
b

(10, 'alpha', [1, 2, 99])

In [28]:
id(b[-1])  # mas o id de b[-1] não mudou

4360618816

👉 **RESUMO:** o que nunca muda em uma tupla são as referências que ela contém. Mas se o valor de um item da tupla mudar, isso muda o valor da tupla.

Em outras palavras: a identidade dos itens em uma tupla nunca muda, mas seus valores podem mudar, dependendo do tipo do item.

Copiar um conteiner abre uma questão: os itens do novo container devem ser cópias dos itens originais, ou podem ser referências para os mesmos itens? Esse é o tema da próxima sessão.